In [1]:
import pandas as pd
import json

In [2]:
lines = []

with open("data/comments2", "r") as f:
    for line in f:
        lines.append(json.loads(line))

In [3]:
lines = pd.DataFrame(lines)

In [4]:
# Get the text data
lines_ = lines[["body_cleaned", "score"]]

In [ ]:
# Strip the text data
lines_["body_cleaned"] = lines_["body_cleaned"].str.strip()

In [6]:
# Remove empty rows
lines_ = lines_[lines_["body_cleaned"] != ""]

In [7]:
# Remove null text or score
lines_ = lines_[lines_["body_cleaned"].notnull()]
lines_ = lines_[lines_["score"].notnull()]

In [8]:
# Add the score to the text data by adding "score: {score of comment}\n" to the beginning of each comment
lines_["body_cleaned"] = "score: " + lines_["score"].astype(str) + "\n" + lines_["body_cleaned"]

In [9]:
# Get all the text data
text = lines_["body_cleaned"].values

In [10]:
# Create huggingface dataset
from datasets import Dataset

dataset = Dataset.from_dict({"text": text})

In [ ]:
dataset.push_to_hub("gmongaras/reddit_political_2019_Feb")